# Projet Python pour la data science 2024 - 2025
*HADDOUCHE Théo, GOURVENNEC Jules, MATHÉ Cyprien, BROCHET Lison*

# Introduction
Depuis les Jeux Olympiques de Paris 2024, un certain nombre de jeunes se sont pris de passion pour le tennis de table et la natation. Cependant, cet engouement s’est heurté au manque d’infrastructure, à leur vétusté et au manque d’encadrant. Cela a de nouveau mis en lumière ce que nous avons appelé les « déserts sportifs ». A l’instar des « déserts médicaux », les « déserts sportifs » sont des zones géographiques souvent rurales, où les infrastructures sportives et les encadrants manquent et où infrastructures existantes sont pour la plupart vétustes et mal équipées. Cela qui constitue un obstacle à la pratique sportive des habitants de ces zones.

Pourtant, il semble exister un consensus sur les bienfaits d’une pratique sportive régulière, en cela qu’elle permet de prévenir un grand nombre de maladies notamment cardiovasculaires ; mais aussi en tant que facteur d’intégration sociale. Aussi, nous avons souhaité questionner cette notion de « désert sportif » sur la base du recensement des équipements sportifs en France. Pour cela, il nous a semblé pertinent d’analyser plusieurs variables telles que la vétusté des infrastructures étudiées, l’offre de sport selon les territoires, l’accessibilité aux infrastructures via des transports en commun.

Notre projet a donc pour objectif de répondre à la problématique suivante : A quel point la situation géographique d’un individu est-elle déterminante dans sa capacité à poursuivre une activité sportive régulière ?


# I/ Récupération et traitement des données

## A. Récupération des données : prise en main de l'API

Les données de la base de données Data ES que nous allons utiliser sont accessibles par une API qui nous permet de n'accéder qu'à quelques valeurs choisies en évitant de tout télécharger chaque fois. Deux modes d'accès sont permis, l'un limite à 100 éléments livrant directement un dataframe, un autre sans limite donne le lien de téléchargement d'un .csv. Le lien se construit dans l'interface du site avec une syntaxe proche du SQL accessible ici : https://equipements.sports.gouv.fr/explore/dataset/data-es/api/?q.

Les fonctions qui permettent d'obtenir les dataframes par ces deux méthodes sont importées et visibles depuis fonctions.py.

In [21]:
import pandas as pd
from fonctions import *

#Un exemple de lien pour obtenir le nombre d'infrastructures par département à l'aide de la méthode group_by et renommé sous la varibale nb_infra
api_url = "https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/records?select=COUNT(inst_numero)%20as%20nb_infra&group_by=dep_nom&order_by=dep_code&limit=-1"
tab = call_api(api_url)
tab.head(3)

dict_keys(['results'])


,dep_nom,nb_infra
0,None,3
1,Ain,4118
2,Aisne,3691


## B. Nettoyage des données

Nos données proviennent d'une API. Pour ce qui est du nettoyage, nous avons remarqué que de nombreux doublons apparaissaient initialement dans la base de données utilisée. Cela s'explique par le fait que plusieurs activités sportives sont proposées au sein d'un même endroit. Par exemple, il est fréquent d'avoir deux lignes pour une piscine comptant deux bassins. Pour remédier à ce problème, nous n'avons gardé qu'une ligne pour chaque lieu puis nous avons créé une nouvelle variable nous permettant de compter le nombre de lignes qu'il y avait avant. 
Autrement, les données étaient plutôt propres toutefois, nous pouvons noter un nombre important de cases n'ayant pas été renseignées.  

Sur le site data.gouv une page est dédiée à cette base, qui contient un url permettant de télécharger la base au format csv, donc nous utilisons directement cet url pour obtenir les données ici. Les lignes qui suivent correspondent à cette opération ainsi qu'à la sélection de nos variables d'intérêt.

Nous allons donc procéder au traitement sur le jeu réduit de paramètres (31/117 disponibles):

inst_numero, inst_nom, dep_nom, dep_code_filled, new_code, new_name, inst_etat, 
inst_date_creation, inst_date_etat, inst_date_valid, inst_acc_handi_bool, 
inst_trans_bool, equip_numero, equip_nom, equip_type_code, 
equip_type_name, equip_type_famille, equip_x, equip_y, 
equip_etat, equip_eclair, equip_douche, equip_sanit, 
equip_pmr_aire, equip_pmr_douche, equip_pmr_sanit, 
equip_travaux_date, equip_service_date, equip_travaux_type, 
equip_energie, equip_prop_type, dens_lib, zrr_simp


In [2]:
#Cette cellule prend environ 1m30 à être exécutée
url = "https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/records?select=inst_numero%2C%20inst_nom%2C%20dep_nom%2C%20dep_code_filled%2C%20new_code%2C%20new_name%2C%20inst_etat%2C%20%20inst_date_creation%2C%20inst_date_etat%2C%20inst_date_valid%2C%20inst_acc_handi_bool%2C%20%20inst_trans_bool%2C%20equip_numero%2C%20equip_nom%2C%20equip_type_code%2C%20%20equip_type_name%2C%20equip_type_famille%2C%20equip_x%2C%20equip_y%2C%20%20equip_etat%2C%20equip_eclair%2C%20equip_douche%2C%20equip_sanit%2C%20%20equip_pmr_aire%2C%20equip_pmr_douche%2C%20equip_pmr_sanit%2C%20%20equip_travaux_date%2C%20equip_service_date%2C%20equip_travaux_type%2C%20%20equip_energie%2C%20equip_prop_type%2C%20dens_lib%2C%20zrr_simp&limit=-1"

recensement = call_csv(url)
recensement.head(3)

/tmp/ipykernel_169945/3045731416.py:5: DtypeWarning: Columns (4,6,57,69,103,105,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url, delimiter=';')


  inst_numero                         inst_nom                  dep_nom  \
0  I042090005  Complexe sportif Daniel Maffren  Alpes-de-Haute-Provence   
1  I042090005  Complexe sportif Daniel Maffren  Alpes-de-Haute-Provence   
2  I042090009               Poney club du Thor  Alpes-de-Haute-Provence   
3  I042090010      Site d'escalade de La Baume  Alpes-de-Haute-Provence   
4  I042090010      Site d'escalade de La Baume  Alpes-de-Haute-Provence   
5  I042090010      Site d'escalade de La Baume  Alpes-de-Haute-Provence   
6  I042090010      Site d'escalade de La Baume  Alpes-de-Haute-Provence   
7  I042090010      Site d'escalade de La Baume  Alpes-de-Haute-Provence   
8  I042090014               Piscine Municipale  Alpes-de-Haute-Provence   
9  I042090015              Boulodrome Val Gele  Alpes-de-Haute-Provence   

  new_code  new_name inst_etat inst_date_creation inst_date_etat  \
0     4209  Sisteron  Attribué         2005-09-22     2021-01-26   
1     4209  Sisteron  Attribué        

Nous remarquons dans le jeu de données, que les lignes ne correspondent pas forcément à un seul site, mais si le site possède plusieurs infrastructures, il est dupliqué autant de fois que nécessaires. Nous avons décidé de ne garder qu'une seule ligne par adresse postale, et de compter combien de fois cette même adresse apparaît dans le jeu de données.

In [3]:
# Création d'une liste qui compte le nombre de doublons par couple (nom d'infrastructure, code de commune) censé représenter une infrastructure unique
doublons = (recensement
            .groupby(['inst_nom', 'new_code'])
            .size()
            .reset_index(name='n')
            .query('n > 0'))

#Suppression des précédents doublons en les remplaçant tous par la première occurence du couple dans le dataset
distinct = recensement.drop_duplicates(subset=['inst_nom', 'new_code'], keep='first')
print("Doublons supprimés : ", len(recensement)-len(distinct))
print("Fréquence de lignes supprimées :", (len(recensement)-len(distinct))/len(recensement))

#Ajout du nombre d'occurence correspondant à chaque couple (nombre de doublons supprimés par infrastructure dans la liste)
df_es = pd.merge(distinct, doublons, on=['inst_nom', 'new_code'], how='outer')

Doublons supprimés :  164548
Fréquence de lignes supprimées : 0.4945064191950762


Près de 50% des lignes du datasets ont été supprimées, le nettoyage était donc bien nécessaire.

## B. Création de nouvelles variables 

# II/ Analyse descriptive et représentation graphique

## A. Description de la base de données utilisée

Des petits histogrammes

## B. Grandes tendances observées

# III/ Modélisation